In [1]:
import numpy as np
import pytorch_kinematics as pk

In [2]:
xml_path = '../descriptions/2_finger_hand_ssss_v2.xml'
chain = pk.build_chain_from_mjcf(xml_path, 'hand')



In [23]:
finger_joint_limit = np.array([[-np.pi / 2, -np.pi / 2, -np.pi / 2, -np.pi / 2],
                                   [np.pi / 2, np.pi / 2, np.pi / 2, np.pi / 2]])

nums = 100000
q_rand_1 = np.random.uniform(finger_joint_limit[0,:], finger_joint_limit[1,:], size=(nums,4))
q_rand_2 = np.random.uniform(finger_joint_limit[0,:], finger_joint_limit[1,:], size=(nums,4))
q_rand = np.concatenate([q_rand_1, q_rand_2], axis=1)

ret = chain.forward_kinematics(q_rand,
                                       end_only=False)

ret0 = chain.forward_kinematics(np.zeros(8),
                                       end_only=False)

In [4]:
ret0.keys()

dict_keys(['hand', 'finger_1', 'MCP_spread_motor_1', 'MCP_motor_1', 'PIP_motor_1', 'DIP_motor_1', 'finger_1_site_1', 'finger_1_tip', 'finger_1_site_2', 'finger_1_site_3', 'finger_1_site_4', 'finger_2', 'MCP_spread_motor_2', 'MCP_motor_2', 'PIP_motor_2', 'DIP_motor_2', 'finger_2_site_1', 'finger_2_tip', 'finger_2_site_2', 'finger_2_site_3', 'finger_2_site_4'])

In [5]:
ret0['finger_1_site_4'].get_matrix()

tensor([[[1.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 1.0000, 0.0000, 0.0725],
         [0.0000, 0.0000, 1.0000, 0.0140],
         [0.0000, 0.0000, 0.0000, 1.0000]]])

In [26]:
color = np.array([[[254,240,217],
                    [253,212,158],
                    [253,187,132],
                    [252,141,89],
                    [227,74,51],
                    [179,0,0]],
                  [[241,238,246],
                    [208,209,230],
                    [166,189,219],
                    [116,169,207],
                    [43,140,190],
                    [4,90,141]]]) / 255

In [28]:
color.shape

(2, 6, 3)

In [33]:
print(color[1,3,:])

[0.45490196 0.6627451  0.81176471]


In [41]:
print(np.array([166,189,219])/255)

[0.65098039 0.74117647 0.85882353]


In [8]:
%matplotlib notebook

iF = 1
joint_names = ['MCP_spread_' + str(iF), 'MCP_' + str(iF), 'PIP_' + str(iF), 'DIP_' + str(iF)]

import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

alpha = 0.5
link_names = [[],[]]

for iF in [1, 2]:
    link_names[iF-1] = ['finger_' + str(iF) +'_tip', 'finger_'+ str(iF) +'_site_1', 'finger_'+ str(iF) +'_site_2', 'finger_'+ str(iF) +'_site_3', 'finger_'+ str(iF) +'_site_4']
    link_positions = []
    for j, link in enumerate(link_names[iF-1]):
        link_position = ret[link].get_matrix()[:, :3, 3].cpu().numpy()
        link_positions.append(link_position)
        ax.scatter(link_position[:, 0], link_position[:, 1], link_position[:, 2], c=tuple(color[iF-1, j, :].flatten()) + (alpha,), s=0.1, marker='o')
    

ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_zlabel('Z (m)')
fig.savefig('reachability_map_2_fingers.jpg', format='jpg', bbox_inches='tight',  pad_inches=0.2, dpi=600)

<IPython.core.display.Javascript object>

/tmp/ipykernel_2968691/3873924012.py:19: UserWarning: *c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
  ax.scatter(link_position[:, 0], link_position[:, 1], link_position[:, 2], c=tuple(color[iF-1, j, :].flatten()) + (alpha,), s=0.1, marker='o')


In [9]:
import xml.etree.ElementTree as ET

mesh_dir = '../descriptions/v2/meshes/'
robot_links = ['base_link_1', 'MCP_spread_motor_1', 'MCP_motor_1','proximal_1','PIP_motor_1', 'middle_1', 'DIP_motor_1', 'distal_1','metacarpal_1']

alpha_robot = 1
colors = [(0.7, 0.7, 0.7, alpha_robot), 
          (0.3, 0.3, 0.3, alpha_robot), 
          (0.3, 0.3, 0.3, alpha_robot),
          (0.7, 0.7, 0.7, alpha_robot), 
          (0.3, 0.3, 0.3, alpha_robot),
          (0.7, 0.7, 0.7, alpha_robot),
          (0.3, 0.3, 0.3, alpha_robot),
          (0, 0.7, 0, alpha_robot),
          (0.7, 0.7, 0.7, alpha_robot)]  



In [10]:
link_names

[['finger_1_tip',
  'finger_1_site_1',
  'finger_1_site_2',
  'finger_1_site_3',
  'finger_1_site_4'],
 ['finger_2_tip',
  'finger_2_site_1',
  'finger_2_site_2',
  'finger_2_site_3',
  'finger_2_site_4']]

In [21]:
import faiss
def remove_abnormal_points(points):
    # 定义 k 值（KNN 中的邻居数）
    k = 5
    
    # 使用 Faiss 构建索引
    index = faiss.IndexFlatL2(points.shape[1])  # 使用 L2 距离计算
    index.add(points)  # 添加点云数据到索引
    
    # 查询每个点的 K 近邻
    distances, indices = index.search(points, k + 1)  # 查询自身时最近距离为0，忽略第一个距离
    
    # 计算每个点的平均距离（忽略第一个距离，即自身距离0）
    mean_distances = distances[:, 1:].mean(axis=1)
    
    # 设置一个阈值，过滤离群点（例如平均距离超过均值加两倍标准差的点）
    threshold = np.mean(mean_distances) + 1.4 * np.std(mean_distances)
    filtered_points = points[mean_distances < threshold]
    
    print(f"原始点数: {len(points)}")
    print(f"过滤后点数: {len(filtered_points)}")

    return filtered_points

In [24]:
import alphashape
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from scipy.spatial import Delaunay
# plot the robot
import trimesh
from mpl_toolkits.mplot3d import Axes3D, art3d

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

replace_link_names = ['finger_1', 'MCP_spread_motor_1','MCP_motor_1','PIP_motor_1','DIP_motor_1' ]
ori_link_names = ['base_link_1', 'metacarpal_1','proximal_1','middle_1','distal_1']

for i, link in enumerate(robot_links):
    if link == 'base_link_1':
        load_link = 'base_link'
    else: 
        load_link = link
        
    mesh = trimesh.load(mesh_dir + load_link + '.stl')
    if link in ori_link_names:
        index_re = ori_link_names.index(link)
        replace_link_name = replace_link_names[index_re]
        pose = ret0[replace_link_name].get_matrix()[0,:,:].cpu().numpy()
    else:
        pose = ret0[link].get_matrix()[0,:,:].cpu().numpy()
        
    mesh.apply_transform(pose)
    vertices = mesh.vertices/1000
    faces = mesh.faces 
    poly3d = [[vertices[vertex] for vertex in face] for face in faces]

    # 创建多边形集合，并将其添加到3D绘图中
    mesh_collection = art3d.Poly3DCollection(poly3d, facecolor=colors[i])
    ax.add_collection3d(mesh_collection)


from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN


ps = []
for iF in [1, 2]:
    for i in range(2):   # range of links
        
        points = np.concatenate([ret[link_names[iF-1][i]].get_matrix()[:, :3, 3].cpu().numpy(),
                                 ret[link_names[iF-1][i+1]].get_matrix()[:, :3, 3].cpu().numpy()], axis=0)
        
        points = remove_abnormal_points(points)

        ps.append(points)
        # 
        # # 使用 alphashape 生成 alpha shape
        alpha = 0.0001 # 可以尝试不同的 alpha 值
        mesh = alphashape.alphashape(points, alpha)
        mesh.export(mesh_dir + str(iF) + str(i) + '.stl')
      
        # 
        # # 绘制点云
        # ax.scatter(points[:, 0], points[:, 1], points[:, 2], s=10, color='blue', alpha=0.1)
        # 绘制 alpha shape 多面体
        faces = np.array(mesh.faces)
        vertices = np.array(mesh.vertices)
        poly3d = [[vertices[vertex] for vertex in face] for face in faces]
        ax.add_collection3d(Poly3DCollection(poly3d, alpha=0.5, color=tuple(color[iF-1, i+2, :].flatten()), edgecolor='none'))

# 设置轴范围
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ps = np.vstack(ps)
min_x, max_x = np.min(ps[:, 0]), np.max(ps[:, 0])
min_y, max_y = np.min(ps[:, 1]), np.max(ps[:, 1])
min_z, max_z = np.min(ps[:, 2]), np.max(ps[:, 2])

# 设置轴范围
ax.set_xlim([min_x, max_x])
ax.set_ylim([min_y, max_y])
ax.set_zlim([min_z, max_z])

# plt.show()
fig.savefig('reachability_map_2_fingers_v2.jpg', format='jpg', bbox_inches='tight',  pad_inches=0.2, dpi=600)


<IPython.core.display.Javascript object>

原始点数: 200000
过滤后点数: 188629
原始点数: 200000
过滤后点数: 178521


原始点数: 200000
过滤后点数: 188736
原始点数: 200000
过滤后点数: 178698


In [13]:
points = np.concatenate([ret[link_names[iF-1][i]].get_matrix()[:, :3, 3].cpu().numpy(),
                                 ret[link_names[iF-1][i+1]].get_matrix()[:, :3, 3].cpu().numpy()], axis=0)
a = Delaunay(points)


In [19]:
points

array([[-0.06438765,  0.15339802,  0.08208616],
       [-0.08189254,  0.05856388,  0.10956756],
       [-0.14313343, -0.00531648, -0.03868315],
       ...,
       [-0.12605143,  0.06001517,  0.03786667],
       [-0.04412298,  0.09731709, -0.03292758],
       [-0.08835173,  0.02669315,  0.06151159]], dtype=float32)

In [20]:
from skimage import measure

verts, faces, normals, values = measure.marching_cubes(points, level=0.5)


ValueError: Input volume should be a 3D numpy array.